In [1]:
# !pip install openpyxl
# !pip install -U scikit-learn
# !pip install nltk
# !pip install --upgrade gensim
# !pip install pyvi

# Cách tiền xử lý dữ liệu bằng Word2Vec

BƯỚC 1: ĐỌC DỮ LIỆU VÀ KIỂM TRA THÔNG SỐ DỮ LIỆU

In [2]:
import pandas as pd
import numpy as np
import openpyxl

Đọc dữ liệu

In [3]:
dataset = r'E:\NCKH\Tài liệu\dataset.xlsx'
dsf = pd.read_excel(dataset)

In [4]:
dsf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2662 entries, 0 to 2661
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Opinion  2662 non-null   object
 1   Labels   2659 non-null   object
dtypes: object(2)
memory usage: 41.7+ KB


In [5]:
dsf.describe()

,Opinion,Labels
count,2662,2659
unique,2543,5
top,Trường đào tạo giao thông đầu ngành của Việt N...,Positive
freq,4,1168


In [6]:
dsf = dsf.dropna()

In [7]:
dsf.describe()

,Opinion,Labels
count,2659,2659
unique,2540,5
top,Trường đào tạo giao thông đầu ngành của Việt N...,Positive
freq,4,1168


In [8]:
dsf = dsf.drop_duplicates(subset=['Opinion'])

In [9]:
dsf.describe()

,Opinion,Labels
count,2540,2540
unique,2540,5
top,"khuôn viên rộng rãi, khang trang, sạch sẽ, mở ...",Positive
freq,1,1103


In [10]:
dsf = dsf.sample(frac=1).reset_index(drop=True)

In [11]:
dsf.describe()

,Opinion,Labels
count,2540,2540
unique,2540,5
top,thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk,Positive
freq,1,1103


In [12]:
dsf.head()

,Opinion,Labels
0,thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk,Neutral
1,UTC Confession sẽ trở thành một điểm đến lý tư...,Positive
2,"Là một nơi đào tạo tốt, thân thiện với tất...",Positive
3,thái độ ko tốt thật. Thôi thì để xem trường xử...,Negative
4,Chúc mừng thành tích của cả thày và trò Trương...,Positive


In [13]:
dsf['Labels'].value_counts()

Positive     1103
Neutral       853
Negative      577
positive        5
Positive        2
Name: Labels, dtype: int64

In [14]:
# Đưa ra danh sách các loại nhãn:
unique_labels : np.ndarray = dsf["Labels"].unique()
unique_labels.sort()
print(unique_labels)

['Negative' 'Neutral' 'Positive' 'Positive ' 'positive']


In [15]:
# Xử lý nhãn sử dụng hàm apply
handle_labels = dsf["Labels"].apply(
    lambda label: label if label in unique_labels[0:3] else unique_labels[2])
dsf = dsf.drop(columns="Labels")
dsf["Labels"] = handle_labels

In [16]:
dsf.describe()

,Opinion,Labels
count,2540,2540
unique,2540,3
top,thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk,Positive
freq,1,1110


In [17]:
dsf['Labels'].value_counts()

Positive    1110
Neutral      853
Negative     577
Name: Labels, dtype: int64

THAY THẾ KÍ TỰ XUỐNG DÒNG, XÓA CÁC DÒNG LINK, TAGS: https, http, #ABC, @NAME, SĐT, ...

Chuyển kí tự xuống dòng thành dấu cách

In [18]:
def clear_endl(text):
    text = text.replace("\n", " ")
    return text

In [19]:
dsf['Remove_endl'] = dsf['Opinion'].apply(lambda x : clear_endl(x))

In [20]:
dsf['Remove_endl']

0           thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
1       UTC Confession sẽ trở thành một điểm đến lý tư...
2       Là một nơi đào tạo tốt, thân thiện với tất...
3       thái độ ko tốt thật. Thôi thì để xem trường xử...
4       Chúc mừng thành tích của cả thày và trò Trương...
                              ...                        
2535    em vào ngành ktxd nhé, anh thấy như một thiên ...
2536    Tuyển dụng Nhân viên Nữ bán hàng thời trang tạ...
2537                             clb có ban tiếp rịu k ạ?
2538        Ra đường mặc đồng phục UTC như khoác long bào
2539    UTC là một trong các trường công lập có mức họ...
Name: Remove_endl, Length: 2540, dtype: object

Xóa bỏ các link, dòng tags,...

In [21]:
import re

In [22]:
def clear_link(text):
    text = re.sub(r'https\S+', " ", text)
    text = re.sub(r'http\S+', " ", text)
    text = re.sub(r'@\w+(\.com(\.vn))?', " ", text)
    text = re.sub(r'#\w+', " ", text)
    text = re.sub(r'\d{10}', " ", text)
    return text

In [23]:
dsf['Remove_endl'] = dsf['Remove_endl'].apply(lambda x: clear_link(x))

In [24]:
dsf['Remove_endl'].head()

0        thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
1    UTC Confession sẽ trở thành một điểm đến lý tư...
2    Là một nơi đào tạo tốt, thân thiện với tất...
3    thái độ ko tốt thật. Thôi thì để xem trường xử...
4    Chúc mừng thành tích của cả thày và trò Trương...
Name: Remove_endl, dtype: object

BỎ CÁC KÝ TỰ ĐẶC BIỆT, EMOJI VÀ KHOẢNG TRẮNG THỪA

Bỏ ký tự đặc biệt

In [25]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [26]:
def remove_punc(text):
    text = re.sub(r'[^a-zA-Z0-9\sÁ-ỹ]', " ", text)
    return text

In [27]:
dsf['Remove_punc'] = dsf['Remove_endl'].apply(lambda x : remove_punc(x))

In [28]:
dsf['Remove_punc'].head()

0        thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
1    UTC Confession sẽ trở thành một điểm đến lý tư...
2    Là một nơi đào tạo tốt  thân thiện với tất...
3    thái độ ko tốt thật  Thôi thì để xem trường xử...
4    Chúc mừng thành tích của cả thày và trò Trương...
Name: Remove_punc, dtype: object

Thay emoji thành khoảng trắng (space)

In [29]:
import re

In [30]:
def remove_emoji(data):
    emoji = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        u"\U000023F0"
                      "]+", re.UNICODE)
    return re.sub(emoji, " ", data)

In [31]:
dsf['Remove_emoji'] = dsf['Remove_punc'].apply(lambda x : remove_emoji(x))

In [32]:
dsf['Remove_emoji'].head()

0        thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
1    UTC Confession sẽ trở thành một điểm đến lý tư...
2    Là một nơi đào tạo tốt  thân thiện với tất...
3    thái độ ko tốt thật  Thôi thì để xem trường xử...
4    Chúc mừng thành tích của cả thày và trò Trương...
Name: Remove_emoji, dtype: object

Chuyển các khoảng trắng về một khoảng trắng

In [33]:
def remove_space(text):
    text= " ".join(text.split())
    return text

In [34]:
dsf['Remove_space'] = dsf['Remove_emoji'].apply(lambda x : remove_space(x))

In [35]:
dsf['Remove_space']

0           thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
1       UTC Confession sẽ trở thành một điểm đến lý tư...
2       Là một nơi đào tạo tốt thân thiện với tất ...
3       thái độ ko tốt thật Thôi thì để xem trường xử ...
4       Chúc mừng thành tích của cả thày và trò Trương...
                              ...                        
2535    em vào ngành ktxd nhé anh thấy như một thiên đ...
2536    Tuyển dụng Nhân viên Nữ bán hàng thời trang tạ...
2537                              clb có ban tiếp rịu k ạ
2538        Ra đường mặc đồng phục UTC như khoác long bào
2539    UTC là một trong các trường công lập có mức họ...
Name: Remove_space, Length: 2540, dtype: object

In [36]:
dsf['Remove_space'][39]

'Ở thời điểm hiện tại việc giảng dạy cũng như đào tạo tại trường đang dần được cải thiện Đội ngũ giảng viên đã chú trọng hơn đến kỹ năng thực hành chứ không chỉ đơn thuần là lý thuyết suông Hy vọng các bạn sinh viên tốt nghiệp từ môi trường này sẽ năng động tự tin và được trang bị đầy đủ kiến thức hơn'

CHUYỂN DỮ LIỆU VỀ KÝ TỰ THƯỜNG, CHUNG FORM, LÀM RÕ CÁC TỪ VIẾT TẮT

Chuyển về ký tự thường

In [37]:
def lower_word(text):
    return text.lower()

In [38]:
dsf['Lower'] = dsf['Remove_space'].apply(lambda x : lower_word(x))

In [39]:
dsf['Lower'].head()

0        thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
1    utc confession sẽ trở thành một điểm đến lý tư...
2    là một nơi đào tạo tốt thân thiện với tất ...
3    thái độ ko tốt thật thôi thì để xem trường xử ...
4    chúc mừng thành tích của cả thày và trò trương...
Name: Lower, dtype: object

Kiểm tra còn hàng trùng không

In [40]:
dsf['Lower'].describe()

count             2540
unique            2497
top       utc mãi đỉnh
freq                 5
Name: Lower, dtype: object

In [41]:
dsf['Lower'] = dsf['Lower'].drop_duplicates()

In [42]:
dsf.describe()

,Opinion,Labels,Remove_endl,Remove_punc,Remove_emoji,Remove_space,Lower
count,2540,2540,2540,2540,2540,2540,2497
unique,2540,3,2540,2537,2537,2507,2497
top,thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk,Positive,thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk,Được đánh giá là có nhiều môn học rất Dễ nh...,Được đánh giá là có nhiều môn học rất Dễ nh...,Đặc biệt Giao thông được mệnh danh là kinh đô ...,thui chúc ae utc nghỉ tết ngắn hạn dui dẻ kkk
freq,1,1110,1,2,2,3,1


In [43]:
dsf = dsf.dropna()

In [44]:
dsf = dsf.sample(frac=1).reset_index(drop=True)

In [45]:
dsf.describe()

,Opinion,Labels,Remove_endl,Remove_punc,Remove_emoji,Remove_space,Lower
count,2497,2497,2497,2497,2497,2497,2497
unique,2497,3,2497,2497,2497,2497,2497
top,Tạch 5 lần GT1 có coi là kỉ lục không,Positive,Tạch 5 lần GT1 có coi là kỉ lục không,Tạch 5 lần GT1 có coi là kỉ lục không,Tạch 5 lần GT1 có coi là kỉ lục không,Tạch 5 lần GT1 có coi là kỉ lục không,tạch 5 lần gt1 có coi là kỉ lục không
freq,1,1084,1,1,1,1,1


In [46]:
dsf['Lower']

0                   tạch 5 lần gt1 có coi là kỉ lục không
1       ám ảnh đầu năm nhất học chính trị pháp luật tư...
2       đk ktx chưa xin đc giấy khai báo thay đổi nhân...
3                            thiếu combo mất nón bh ở cs1
4       chưa cảm thấy thực sự hài lòng về cách làm việ...
                              ...                        
2492    chả có gì mà chanh cái cả cả 2 ông đều sai 1 b...
2493                                          dễ tính nhé
2494    nên phân biệt rõ ràng giữa gtvt bình thạnh và ...
2495                           học xưởng tâm trí trên mây
2496    ngôi trường rộng ơi là rộng đến đây một lần ch...
Name: Lower, Length: 2497, dtype: object

NỐI TỪ GHÉP (TỪ TIẾNG VIỆT)

In [1]:
import pyvi
from pyvi import ViTokenizer

In [2]:
def join_word(text):
    if not isinstance(text, (float, int)):
        sentence_tokenized = ViTokenizer.tokenize(text)
        words = sentence_tokenized.split()
        return words
    else:
        return None


In [3]:
simple_text = "Trường đại học giao thông vận tải là một trong những trường top kỹ thuật hàng đầu Việt Nam"
print (simple_text)
join_text = join_word(simple_text)
print (join_text)

Trường đại học giao thông vận tải là một trong những trường top kỹ thuật hàng đầu Việt Nam
['Trường', 'đại_học', 'giao_thông', 'vận_tải', 'là', 'một', 'trong', 'những', 'trường', 'top', 'kỹ_thuật', 'hàng_đầu', 'Việt_Nam']


In [49]:
dsf['Join_word'] = dsf['Lower'].apply(lambda x : join_word(x))

In [50]:
dsf['Join_word']

0         [tạch, 5, lần, gt1, có, coi, là, kỉ_lục, không]
1       [ám_ảnh, đầu, năm, nhất, học, chính_trị, pháp_...
2       [đk, ktx, chưa, xin, đc, giấy, khai_báo, thay_...
3                    [thiếu, combo, mất, nón, bh, ở, cs1]
4       [chưa, cảm_thấy, thực_sự, hài_lòng, về, cách, ...
                              ...                        
2492    [chả, có, gì, mà, chanh, cái, cả, cả, 2, ông, ...
2493                                       [dễ_tính, nhé]
2494    [nên, phân_biệt, rõ_ràng, giữa, gtvt, bình_thạ...
2495                     [học, xưởng, tâm_trí, trên, mây]
2496    [ngôi, trường, rộng, ơi, là, rộng, đến, đây, m...
Name: Join_word, Length: 2497, dtype: object

THAY THẾ CÁC TỪ VIẾT TẮT

In [51]:
file = open(
    r"E:\NCKH\Code\repairword.txt", encoding="utf-8")

In [52]:
repair_words_list = dict(line.strip().split(":") for line in file)

In [53]:
def repair_words(text):
    for i in range(len(text)):
        if text[i] in repair_words_list:
            text[i] = repair_words_list[text[i]]

    # ghép các từ thành câu mới
    return text

In [54]:
abc = ["j", "jkae", "sv", "svien", "kkk", "ql", "qldt", "a", "ac", "aaaa", "ae", "k"]
abc = repair_words(abc)
abc

['gì',
 'jkae',
 'sinh_viên',
 'sinh_viên',
 'kkk',
 'quản_lý',
 'quản_lý_đào_tạo',
 'anh',
 'anh_chị',
 'aaaa',
 'anh_em',
 'không']

In [55]:
dsf['Repair_words'] = dsf['Join_word'].apply(lambda x : repair_words(x))

In [346]:
dsf['Repair_words']

0       [đã, tìm, ra, những, chủ_nhân, của, chai, nước...
1       [mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc...
2       [tôi, thật_sự, bất_ngờ, trước, sự, hoành_tráng...
3                                         [k62, giãy_nảy]
4       [tiếng, anh, thi, đầu_vào, là, không, bắt_buộc...
                              ...                        
2492    [cậu, có, từng, muốn, quay, lại, với, tớ, khôn...
2493     [kinh_nghiệm, là, nên, chọn, gvhd, cho, kỹ, nhé]
2494    [cái, gì, cảm_thấy, khó, quá, thì, nên, học, l...
2495                                           [chạy, đi]
2496    [wifi, của, trường, trừ, tòa, a8, anh_em, với,...
Name: Repair_words, Length: 2497, dtype: object

XOÁ CÁC TỪ DỪNG

In [347]:
file_vn = open(
    r"E:\NCKH\Code\stopwords.txt", encoding="utf-8")

In [348]:
stopwords_viet_list = file_vn.read()

In [349]:
stopwords_viet_list = stopwords_viet_list.split("\n")

In [350]:
stopwords_viet_list

['à',
 'ạ',
 'bị',
 'bởi',
 'bao_giờ',
 'cả',
 'các',
 'cái',
 'cần',
 'càng',
 'chỉ',
 'chiếc',
 'chứ',
 'chưa',
 'có',
 'có_thể',
 'cơ_mà',
 'cứ',
 'của',
 'cùng',
 'cũng',
 'cmn',
 'chắc',
 'cỡ',
 'cạch',
 'dọc',
 'dãy',
 'dạo',
 'đã',
 'đang',
 'đây',
 'để',
 'đến_nỗi',
 'đúng_ra',
 'đều',
 'điều',
 'do',
 'đó',
 'được',
 'dưới',
 'đành',
 'dành',
 'đâu_đây',
 'gì',
 'giồi',
 'ghê',
 'hả',
 'j',
 'khi',
 'không_chỉ',
 'hoài',
 'là',
 'lại',
 'lên',
 'lúc',
 'lun',
 'luon',
 'luôn',
 'loèn',
 'lâu_lâu',
 'mà',
 'mỗi',
 'một_cách',
 'mới_phải',
 'mấy',
 'nào',
 'nhỉ',
 'này',
 'nên',
 'nếu',
 'ngay',
 'nha',
 'nhé',
 'như',
 'nhưng',
 'những',
 'nơi',
 'nữa',
 'như_vậy',
 'như_thế',
 'ngặt',
 'oh',
 'òi',
 'ồ',
 'ôi',
 'ối',
 'ơ',
 'ờ',
 'ớ',
 'ơi',
 'phải',
 'qua',
 'quá',
 'ra',
 'rằng',
 'rồi',
 'ra_mặt',
 'sời',
 'sau',
 'sẽ',
 'so',
 'sự',
 'thế',
 'tại',
 'theo',
 'thì',
 'trong',
 'trên',
 'trước',
 'từ',
 'từng',
 'tí',
 't_kb',
 'thứ',
 'uầy',
 'ư',
 'ừ',
 'ừm',
 'và',
 'vẫn

In [351]:
def remove_stopword(text):
    if text is not None:
        result = list()
        for word in text:
            if (word not in stopwords_viet_list):
                result.append(word)
        return result

In [352]:
abc = ["ra", "rang", "ráng", "so", "song"]
abc = remove_stopword(abc)
abc

['rang', 'ráng', 'song']

In [353]:
dsf['Remove_stopword'] = dsf["Repair_words"].apply(lambda x : remove_stopword(x))

In [354]:
dsf['Remove_stopword']

0       [tìm, chủ_nhân, chai, nước, kẹt, làm, anh_em, ...
1       [mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc...
2       [tôi, thật_sự, bất_ngờ, hoành_tráng, trường_họ...
3                                         [k62, giãy_nảy]
4       [tiếng, anh, thi, đầu_vào, không, bắt_buộc, đâ...
                              ...                        
2492    [cậu, muốn, quay, tớ, không, cậu, đừng, hiểu, ...
2493                   [kinh_nghiệm, chọn, gvhd, cho, kỹ]
2494              [cảm_thấy, khó, học, lần, 2, cho, hiểu]
2495                                           [chạy, đi]
2496    [wifi, trường, trừ, tòa, a8, anh_em, máy, bán_...
Name: Remove_stopword, Length: 2497, dtype: object

In [355]:
dsf.head()

,Opinion,Labels,Remove_endl,Remove_punc,Remove_emoji,Remove_space,Lower,Join_word,Repair_words,Remove_stopword
0,Đã tìm ra những chủ nhân của chai nước bị kẹt ...,Neutral,Đã tìm ra những chủ nhân của chai nước bị kẹt ...,Đã tìm ra những chủ nhân của chai nước bị kẹt ...,Đã tìm ra những chủ nhân của chai nước bị kẹt ...,Đã tìm ra những chủ nhân của chai nước bị kẹt ...,đã tìm ra những chủ nhân của chai nước bị kẹt ...,"[đã, tìm, ra, những, chủ_nhân, của, chai, nước...","[đã, tìm, ra, những, chủ_nhân, của, chai, nước...","[tìm, chủ_nhân, chai, nước, kẹt, làm, anh_em, ..."
1,mọi người nhớ đeo khẩu trang uống thuốc ăn uốn...,Neutral,mọi người nhớ đeo khẩu trang uống thuốc ăn uốn...,mọi người nhớ đeo khẩu trang uống thuốc ăn uốn...,mọi người nhớ đeo khẩu trang uống thuốc ăn uốn...,mọi người nhớ đeo khẩu trang uống thuốc ăn uốn...,mọi người nhớ đeo khẩu trang uống thuốc ăn uốn...,"[mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc...","[mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc...","[mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc..."
2,Tôi thật sự bất ngờ trước sự hoành tráng...,Positive,Tôi thật sự bất ngờ trước sự hoành tráng...,Tôi thật sự bất ngờ trước sự hoành tráng...,Tôi thật sự bất ngờ trước sự hoành tráng...,Tôi thật sự bất ngờ trước sự hoành tráng...,tôi thật sự bất ngờ trước sự hoành tráng...,"[tôi, thật_sự, bất_ngờ, trước, sự, hoành_tráng...","[tôi, thật_sự, bất_ngờ, trước, sự, hoành_tráng...","[tôi, thật_sự, bất_ngờ, hoành_tráng, trường_họ..."
3,K62 giãy nảy :<<,Negative,K62 giãy nảy :<<,K62 giãy nảy,K62 giãy nảy,K62 giãy nảy,k62 giãy nảy,"[k62, giãy_nảy]","[k62, giãy_nảy]","[k62, giãy_nảy]"
4,Tiếng Anh thi đầu vào là không bắt buộc đâu bé...,Neutral,Tiếng Anh thi đầu vào là không bắt buộc đâu bé...,Tiếng Anh thi đầu vào là không bắt buộc đâu bé...,Tiếng Anh thi đầu vào là không bắt buộc đâu bé...,Tiếng Anh thi đầu vào là không bắt buộc đâu bé...,tiếng anh thi đầu vào là không bắt buộc đâu bé...,"[tiếng, anh, thi, đầu_vào, là, không, bắt_buộc...","[tiếng, anh, thi, đầu_vào, là, không, bắt_buộc...","[tiếng, anh, thi, đầu_vào, không, bắt_buộc, đâ..."


In [356]:
dff = pd.DataFrame()
dff['repair_words'] = dsf['Repair_words']
dff['labels'] = dsf['Labels']
dff.to_excel('nremove_stopwords.xlsx', index = False)

In [357]:
# dsf['Repair_words'].to_excel('nremove_stopwords.xlsx', index=False)

### WORD2VEC

In [358]:
import gensim
# datasets = r'E:\NCKH\Code\result.xlsx'
result_file = dsf['Remove_stopword']

In [359]:
gensim.utils.simple_preprocess("🔥🔥🔥 BÙNG NỔ DATA CENTER CUỐI NĂM 2017🔥🔥🔥,List job Full option dành cho sinh viên NĂM CUỐI, MỚI TỐT NGHIỆP tới cả các bạn ĐÃ CÓ KINH NGHIỆM 1 năm trở lên🤔Nắng đã có DATA CENTER🤔Mưa đã có DATA CENTER🤔Việc của bạn chỉ là vào phòng điều hòa và chuyên tâm công tác thôi nhé01. Cộng tác viên vận hành Điện Data center✅ KHÔNG yêu cầu kinh nghiệm, không yêu cầu giới tínhCHỈ CẦN✅ bạn có kiến thức về hệ thống điện, có thể làm việc Full-time✅ ƯU TIÊN sinh viên năm cuối hoặc mới tốt nghiệp02. Nhân viên viện hành hệ thống Data center🔆 Nam giới, độ tuổi 25-35🔆 Có kinh nghiệm xây dựng, triển khai đấu nối cáp điện hạ thế.🔆 Có kiến thức, kinh nghiệm chuyên sâu về TBA, MFĐ, UPS, Điều hòa chính xác… là một lợi thế.🔆 Có khả năng đi công tácCV gửi về: PhuongBB5 @fpt. com. vn<3")

['bùng',
 'nổ',
 'data',
 'center',
 'cuối',
 'năm',
 'list',
 'job',
 'full',
 'option',
 'dành',
 'cho',
 'sinh',
 'viên',
 'năm',
 'cuối',
 'mới',
 'tốt',
 'nghiệp',
 'tới',
 'cả',
 'các',
 'bạn',
 'đã',
 'có',
 'kinh',
 'nghiệm',
 'năm',
 'trở',
 'lên',
 'nắng',
 'đã',
 'có',
 'data',
 'center',
 'mưa',
 'đã',
 'có',
 'data',
 'center',
 'việc',
 'của',
 'bạn',
 'chỉ',
 'là',
 'vào',
 'phòng',
 'điều',
 'hòa',
 'và',
 'chuyên',
 'tâm',
 'công',
 'tác',
 'thôi',
 'nhé',
 'cộng',
 'tác',
 'viên',
 'vận',
 'hành',
 'điện',
 'data',
 'center',
 'không',
 'yêu',
 'cầu',
 'kinh',
 'nghiệm',
 'không',
 'yêu',
 'cầu',
 'giới',
 'tínhchỉ',
 'cần',
 'bạn',
 'có',
 'kiến',
 'thức',
 'về',
 'hệ',
 'thống',
 'điện',
 'có',
 'thể',
 'làm',
 'việc',
 'full',
 'time',
 'ưu',
 'tiên',
 'sinh',
 'viên',
 'năm',
 'cuối',
 'hoặc',
 'mới',
 'tốt',
 'nghiệp',
 'nhân',
 'viên',
 'viện',
 'hành',
 'hệ',
 'thống',
 'data',
 'center',
 'nam',
 'giới',
 'độ',
 'tuổi',
 'có',
 'kinh',
 'nghiệm',
 'xây',
 'dựn

In [360]:
model = gensim.models.Word2Vec(
    vector_size=100,
    window = 6,
    min_count = 1,
    workers=4,
    epochs=30
)

03:08:55 gensim.utils INFO: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-05-13T03:08:55.555970', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]


In [361]:
model.build_vocab(result_file, progress_per=2000)

03:08:55 gensim.models.word2vec INFO: collecting all words and their counts
		at Line 582 [scan_vocab() in word2vec.py, word2vec]
03:08:55 gensim.models.word2vec INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:08:55 gensim.models.word2vec INFO: PROGRESS: at sentence #2000, processed 31931 words, keeping 4747 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:08:55 gensim.models.word2vec INFO: collected 5342 word types from a corpus of 40712 raw words and 2497 sentences
		at Line 588 [scan_vocab() in word2vec.py, word2vec]
03:08:55 gensim.models.word2vec INFO: Creating a fresh vocabulary
		at Line 637 [prepare_vocab() in word2vec.py, word2vec]
03:08:55 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 5342 unique words (100.00% of original 5342, drops 0)', 'datetime': '2023-05-13T03:08:55.713599', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6

In [362]:
model.epochs

30

In [363]:
model.corpus_count

2497

In [364]:
model.wv.index_to_key

['trường',
 'không',
 'học',
 'mình',
 'em',
 'bạn',
 'cho',
 'sinh_viên',
 'đi',
 'nhiều',
 'năm',
 'còn',
 'giao_thông',
 'ở',
 '1',
 'rất',
 'cô',
 'về',
 'một',
 'đại_học',
 'người',
 'thầy',
 'đến',
 'tôi',
 'cậu',
 'làm',
 'utc',
 'xe',
 'vận_tải',
 'biết',
 'thi',
 '2',
 'đào_tạo',
 'ngành',
 'điểm',
 'anh',
 'thôi',
 'mới',
 'thấy',
 'hỏi',
 'nhất',
 'tốt',
 'lớp',
 'lắm',
 'phòng',
 'hay',
 'ngày',
 'hơn',
 '3',
 'giao_thông_vận_tải',
 'nói',
 'ai',
 'đâu',
 'xin',
 'muốn',
 'đẹp',
 'nó',
 'môn',
 'chất_lượng',
 'nhà',
 'khác',
 'lần',
 '4',
 'cao',
 'cơ_sở',
 'giờ',
 'nhau',
 'tiền',
 'yêu',
 'đấy',
 'gái',
 'hết',
 'ấy',
 'mọi',
 'học_phí',
 'trai',
 'kỹ_thuật',
 'nay',
 'bài',
 't',
 'khá',
 'đầu',
 'đường',
 'nam',
 'kiến_thức',
 'bằng',
 'con',
 '5',
 'gửi',
 'dễ',
 'gần',
 'khoa',
 'mất',
 'dạy',
 'tớ',
 'đúng',
 'kinh_tế',
 'toàn',
 'sao',
 'bên',
 'lấy',
 'ký_túc_xá',
 'nhận',
 'nhà_trường',
 '7',
 'thích',
 'tìm',
 'thật',
 'giấy',
 'thêm',
 'thời_gian',
 'tháng',
 'm

In [365]:
model.train(result_file, total_examples=model.corpus_count, epochs = model.epochs)

03:08:56 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'training model with 4 workers on 5342 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=6 shrink_windows=True', 'datetime': '2023-05-13T03:08:56.252133', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:08:56 gensim.models.word2vec INFO: EPOCH 0: training on 40712 raw words (35011 effective words) took 0.0s, 1069182 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:08:56 gensim.models.word2vec INFO: EPOCH 1: training on 40712 raw words (35147 effective words) took 0.0s, 1596589 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:08:56 gensim.models.word2vec INFO: EPOCH 2: training on 40712 raw words (34950 effective words) took 0.0s, 1205002 effective words/s
	

(1051237, 1221360)

In [366]:
model.wv.most_similar("cô")

[('thầy', 0.9505864381790161),
 ('tỉ_mỉ', 0.9498308300971985),
 ('giọng', 0.9418643116950989),
 ('phát_biểu', 0.9406448602676392),
 ('hiền', 0.9346006512641907),
 ('tế', 0.9301651120185852),
 ('gắt_gao', 0.9296017289161682),
 ('giảng', 0.9285761117935181),
 ('ngược_lại', 0.9230173826217651),
 ('6đ', 0.913692057132721)]

In [367]:
model.wv.similarity(w1 = "trọ", w2 = "thầy")

0.104421854

In [368]:
model.wv.similarity(w1 = "trọ", w2 = "ký_túc_xá")

0.8227938

In [369]:
model.wv[0]

array([-0.24123944, -0.01319854, -0.39609438,  0.59385455, -0.85940295,
       -0.49851134,  0.5032343 ,  0.9631318 , -0.3167092 , -0.6437328 ,
       -0.16523984, -0.16773573, -0.6143894 , -0.10609626,  0.06919124,
       -0.82965535,  0.5098542 , -0.75360197, -0.3602913 , -1.1080005 ,
        0.34796798,  1.0791149 ,  0.6699066 , -0.2355005 ,  0.16564809,
        0.1394569 , -1.1398895 ,  0.23273402,  0.03655605,  0.17254362,
        0.10183131,  0.17604774, -0.22726078, -0.11730412, -0.4749573 ,
       -0.09674133, -0.5318355 , -0.42033818, -0.21054325, -1.09927   ,
       -0.69855314,  0.12423498, -0.25070432, -0.39685905,  0.4086218 ,
       -0.4695271 , -0.37763348,  0.11186483,  0.33509523,  1.1334002 ,
        0.02894248, -0.80311084, -0.7301134 , -0.9330631 ,  0.38246801,
       -0.288239  ,  0.5250163 ,  0.4432703 , -0.38957253,  0.68324715,
       -0.36763456, -0.095604  ,  0.42441258, -0.12398258, -0.40725484,
        0.30734554, -0.38321903,  0.8711498 , -0.42281252,  0.98

In [370]:
model.save('model.bin')

03:08:58 gensim.utils INFO: Word2Vec lifecycle event {'fname_or_handle': 'model.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-13T03:08:58.087238', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:08:58 gensim.utils INFO: not storing attribute cum_table
		at Line 710 [_save_specials() in utils.py, utils]
03:08:58 gensim.utils INFO: saved model.bin
		at Line 618 [_smart_save() in utils.py, utils]


In [371]:
#  Tạo DataFrame từ các vecto đầu ra của các câu
df = pd.DataFrame(columns=[str(i) for i in range(100)])
sentences = dsf['Remove_stopword']
for sentence in sentences:
    sentence_vector = []
    for word in sentence:
        try:
            word_vector = model.wv[word]
            sentence_vector.append(word_vector)
        except KeyError:
            pass
    if sentence_vector:
        sentence_vector = pd.DataFrame(sentence_vector)
        sentence_vector = sentence_vector.mean()
        df = df.append(pd.Series(sentence_vector.values, index=df.columns), ignore_index=True)

C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\1972757989.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(sentence_vector.values, index=df.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\1972757989.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(sentence_vector.values, index=df.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\1972757989.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.Series(sentence_vector.values, index=df.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\1972757989.py:15: FutureWarning: The frame.append method is deprecated and will be removed f

In [372]:
# df = pd.DataFrame(model.wv.vectors, index=model.wv.index_to_key)

In [373]:
df['Labels'] = dsf['Labels']

In [374]:
df

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Labels
0,-0.192544,0.294876,0.030149,-0.084672,0.027703,-0.554401,0.283903,0.458641,-0.294528,-0.024376,...,0.205307,0.039121,0.037150,0.613017,0.156718,0.125601,-0.154710,0.019623,0.027649,Neutral
1,-0.327397,0.270287,-0.016242,0.135562,0.247240,-0.507962,-0.022827,0.451834,-0.471072,-0.054671,...,0.301554,0.077708,0.192793,0.482929,0.000004,0.220646,-0.239999,-0.142305,-0.251221,Neutral
2,-0.270892,0.216580,0.075208,0.073458,0.006567,-0.525929,0.032569,0.620475,-0.370090,-0.480970,...,0.134678,0.010907,0.176982,0.412090,0.319371,0.167588,-0.290025,0.107027,-0.184174,Positive
3,-0.184668,0.143689,0.033227,-0.004467,0.064010,-0.266581,0.056624,0.263338,-0.254738,0.023261,...,0.133132,0.040271,0.080003,0.365577,0.117735,0.028094,-0.083292,-0.155704,-0.144196,Negative
4,-0.063730,0.247706,0.219824,-0.082596,0.061695,-0.437401,0.154379,0.536642,-0.104653,-0.183549,...,0.246913,0.064954,0.077668,0.533340,0.334954,-0.211686,-0.117453,-0.125357,-0.198595,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,-0.680350,0.242165,-0.232829,-0.024140,0.051664,-0.881266,-0.102357,0.668268,-0.780829,-0.192114,...,0.228436,0.240018,0.365059,0.634581,-0.164992,0.532896,-0.412166,-0.254913,-0.736782,Positive
2492,-0.098898,0.206614,0.180588,0.023986,0.072877,-0.292906,-0.028978,0.511606,-0.387329,-0.289607,...,0.134529,-0.038718,-0.015749,0.332202,0.175834,0.050695,-0.118427,-0.047801,-0.097990,Neutral
2493,-0.294821,0.131104,0.081792,-0.612859,0.172110,-0.833870,-0.206103,0.578491,-0.436125,-0.508584,...,0.155791,0.217697,0.310930,0.541360,0.319445,0.170984,-0.217526,-0.546210,-1.243511,Neutral
2494,-0.397622,0.460045,-0.073084,-0.088680,0.649215,-0.681673,0.141231,0.339920,-0.544972,0.142987,...,0.606587,0.436140,0.059055,0.944417,0.718919,0.035315,-0.228650,-0.671325,-0.660711,Positive


In [375]:
df.to_excel('result.xlsx', index=False)

In [376]:
dsf["Remove_stopword"].describe()

count            2497
unique           2490
top       [tuyệt_vời]
freq                3
Name: Remove_stopword, dtype: object

### WORD2VEC KHÔNG XÓA TỪ DỪNG

In [377]:
result_files = dsf["Repair_words"]

In [378]:
models = gensim.models.Word2Vec(
    vector_size=100,
    window = 6,
    min_count = 1,
    workers=4,
    epochs=30
)

03:09:14 gensim.utils INFO: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-05-13T03:09:14.784346', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]


In [379]:
models.build_vocab(result_files, progress_per=2000)

03:09:14 gensim.models.word2vec INFO: collecting all words and their counts
		at Line 582 [scan_vocab() in word2vec.py, word2vec]
03:09:14 gensim.models.word2vec INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:09:14 gensim.models.word2vec INFO: PROGRESS: at sentence #2000, processed 43078 words, keeping 4870 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:09:14 gensim.models.word2vec INFO: collected 5468 word types from a corpus of 55129 raw words and 2497 sentences
		at Line 588 [scan_vocab() in word2vec.py, word2vec]
03:09:14 gensim.models.word2vec INFO: Creating a fresh vocabulary
		at Line 637 [prepare_vocab() in word2vec.py, word2vec]
03:09:14 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 5468 unique words (100.00% of original 5468, drops 0)', 'datetime': '2023-05-13T03:09:14.992916', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6

In [380]:
models.train(result_file, total_examples=models.corpus_count, epochs = models.epochs)

03:09:15 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'training model with 4 workers on 5468 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=6 shrink_windows=True', 'datetime': '2023-05-13T03:09:15.291523', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:09:15 gensim.models.word2vec INFO: EPOCH 0: training on 40712 raw words (36387 effective words) took 0.0s, 976203 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:09:15 gensim.models.word2vec INFO: EPOCH 1: training on 40712 raw words (36363 effective words) took 0.0s, 1089910 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:09:15 gensim.models.word2vec INFO: EPOCH 2: training on 40712 raw words (36395 effective words) took 0.0s, 1072198 effective words/s
		

(1091327, 1221360)

In [381]:
models.wv[0]

array([ 7.80173957e-01, -1.49901271e-01,  8.24681446e-02, -8.38721320e-02,
        2.44446069e-01, -6.94747448e-01,  1.46466565e+00,  1.20523751e+00,
        3.74900788e-01, -6.78215325e-01,  3.72669309e-01, -4.01829809e-01,
       -1.58310950e-01,  6.64069891e-01, -8.26640844e-01, -1.65102750e-01,
       -9.62455645e-02, -6.44886315e-01, -5.90661108e-01, -9.71230745e-01,
        7.16394067e-01,  1.07001700e-01,  4.23320353e-01,  6.13584295e-02,
        5.61087847e-01, -2.73229659e-01, -9.69559848e-01,  8.11502561e-02,
       -7.35908389e-01,  2.20755413e-01,  4.91856009e-01, -4.36417997e-01,
        1.15654063e+00, -3.72028202e-01, -4.93808776e-01,  5.77870488e-01,
        1.99693501e-01, -2.46800452e-01, -1.65830106e-01, -7.16955245e-01,
       -1.42019725e+00,  2.45238498e-01, -4.63274986e-01, -3.85986835e-01,
        9.67683911e-01,  1.89046562e-01, -1.22693646e+00,  1.62240304e-02,
       -1.10837184e-02,  1.78876445e-01, -5.28372109e-01,  6.13823771e-01,
        5.96698046e-01, -

In [382]:
models.wv.most_similar("cô")

[('tỉ_mỉ', 0.9444979429244995),
 ('thầy', 0.9377554655075073),
 ('giọng', 0.935539186000824),
 ('hiền', 0.9256882667541504),
 ('phát_biểu', 0.9213272333145142),
 ('giảng', 0.9173597693443298),
 ('cuộc_gọi', 0.9118295311927795),
 ('gắt_gao', 0.8975959420204163),
 ('hương', 0.8932589888572693),
 ('khó_tính', 0.8868985772132874)]

In [383]:
models.save('model_sw.bin')

03:09:16 gensim.utils INFO: Word2Vec lifecycle event {'fname_or_handle': 'model_sw.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-13T03:09:16.943589', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:09:16 gensim.utils INFO: not storing attribute cum_table
		at Line 710 [_save_specials() in utils.py, utils]
03:09:16 gensim.utils INFO: saved model_sw.bin
		at Line 618 [_smart_save() in utils.py, utils]


In [384]:
#  Tạo DataFrame từ các vecto đầu ra của các câu
ndf = pd.DataFrame(columns=[str(i) for i in range(100)])
sentences = dsf["Repair_words"]
for sentence in sentences:
    sentence_vector = []
    for word in sentence:
        try:
            word_vector = models.wv[word]
            sentence_vector.append(word_vector)
        except KeyError:
            pass
    if sentence_vector:
        sentence_vector = pd.DataFrame(sentence_vector)
        sentence_vector = sentence_vector.mean()
        ndf = ndf.append(pd.Series(sentence_vector.values, index=ndf.columns), ignore_index=True)

C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\256369258.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndf = ndf.append(pd.Series(sentence_vector.values, index=ndf.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\256369258.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndf = ndf.append(pd.Series(sentence_vector.values, index=ndf.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\256369258.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ndf = ndf.append(pd.Series(sentence_vector.values, index=ndf.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\256369258.py:15: FutureWarning: The frame.append method is deprecated and will be remo

In [385]:
ndf['Labels'] = dsf['Labels']

In [386]:
ndf

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Labels
0,-0.119068,0.033960,0.033688,0.252632,0.031488,-0.210780,0.076524,0.279842,0.085530,0.164102,...,-0.021321,0.308013,-0.120268,0.385913,0.224953,-0.125305,0.139199,-0.070232,-0.024415,Neutral
1,-0.184615,0.260926,0.071290,0.306071,0.190597,-0.318380,-0.137756,0.327745,-0.032246,-0.023104,...,0.030158,0.194436,-0.109371,0.446313,0.218037,0.291255,-0.077963,-0.123999,-0.124094,Neutral
2,0.094723,0.194646,0.191446,0.228309,0.132649,-0.499585,0.151298,0.402553,0.130694,-0.181887,...,0.204480,0.146234,-0.290805,0.369949,0.061939,0.150950,-0.210698,0.201992,-0.155340,Positive
3,-0.141364,0.064475,0.030685,0.194673,0.108662,-0.184964,-0.016368,0.129672,0.010648,0.034289,...,0.126981,0.156535,-0.015637,0.250536,0.140385,0.033072,0.035951,-0.002448,-0.094805,Negative
4,0.024245,0.104531,-0.056140,0.253684,0.143140,-0.474647,0.161606,0.261300,-0.006065,-0.087465,...,0.269728,0.020503,-0.028578,0.534430,0.036552,-0.079000,-0.203719,0.100937,-0.196009,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,0.105442,0.311902,0.190535,0.280049,0.423015,-0.157738,-0.317618,0.589958,-0.120771,-0.030176,...,0.182918,0.583691,-0.142162,0.619493,0.435444,0.181603,-0.110909,-0.167664,-0.143445,Neutral
2493,-0.148817,0.214579,0.009800,0.209970,-0.071757,-0.367997,0.015628,0.153964,-0.008622,-0.029820,...,0.281459,0.035628,-0.023610,0.151282,0.084772,0.192042,-0.045927,0.037952,-0.033127,Neutral
2494,0.082124,-0.021427,0.110445,0.422655,0.334112,-0.315480,-0.295131,0.479695,-0.030351,0.052719,...,0.117232,0.292206,0.274286,0.555046,0.340913,0.277078,0.047824,-0.107181,-0.493701,Positive
2495,-0.666406,-0.160226,0.153856,0.683043,0.328123,-0.704737,0.208857,0.373633,0.258872,0.353276,...,0.015626,0.456328,0.244802,1.058728,0.785499,0.411663,0.282564,-0.075521,-0.779652,Negative


In [387]:
ndf.to_excel('nresult.xlsx', index=False)

### Thêm từ điển không có từ dừng

In [388]:
from libs.utils import *

In [389]:
print(dsf.iloc[0])
type(dsf["Remove_stopword"].iloc[0])

Opinion            Đã tìm ra những chủ nhân của chai nước bị kẹt ...
Labels                                                       Neutral
Remove_endl        Đã tìm ra những chủ nhân của chai nước bị kẹt ...
Remove_punc        Đã tìm ra những chủ nhân của chai nước bị kẹt ...
Remove_emoji       Đã tìm ra những chủ nhân của chai nước bị kẹt ...
Remove_space       Đã tìm ra những chủ nhân của chai nước bị kẹt ...
Lower              đã tìm ra những chủ nhân của chai nước bị kẹt ...
Join_word          [đã, tìm, ra, những, chủ_nhân, của, chai, nước...
Repair_words       [đã, tìm, ra, những, chủ_nhân, của, chai, nước...
Remove_stopword    [tìm, chủ_nhân, chai, nước, kẹt, làm, anh_em, ...
Name: 0, dtype: object


list

Tìm kiếm các câu không có dữ liệu:

In [390]:
dsf[{"Labels", "Remove_stopword"}]

C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\2827014672.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dsf[{"Labels", "Remove_stopword"}]


,Labels,Remove_stopword
0,Neutral,"[tìm, chủ_nhân, chai, nước, kẹt, làm, anh_em, ..."
1,Neutral,"[mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc..."
2,Positive,"[tôi, thật_sự, bất_ngờ, hoành_tráng, trường_họ..."
3,Negative,"[k62, giãy_nảy]"
4,Neutral,"[tiếng, anh, thi, đầu_vào, không, bắt_buộc, đâ..."
...,...,...
2492,Neutral,"[cậu, muốn, quay, tớ, không, cậu, đừng, hiểu, ..."
2493,Neutral,"[kinh_nghiệm, chọn, gvhd, cho, kỹ]"
2494,Positive,"[cảm_thấy, khó, học, lần, 2, cho, hiểu]"
2495,Negative,"[chạy, đi]"


Tạo từ điển và danh sách nhãn:

In [56]:
corpus: list[list[str]] = [doc for doc in dsf["Remove_stopword"]]
labels: list[str] = dsf["Labels"].to_list()


KeyboardInterrupt



In [392]:
from libs.feature_vocabulary import feature_vocabulary_transfer
feature_vocabulary_trs = feature_vocabulary_transfer("gini")
feature_vocabulary = feature_vocabulary_trs.fit_transform(corpus, labels, 0.3, 3)

In [393]:
feature_vocabulary

{'00': 'Neutral',
 '12h': 'Neutral',
 '14': 'Neutral',
 '14h30': 'Neutral',
 '17h30': 'Neutral',
 '18h': 'Neutral',
 '18h30': 'Neutral',
 '1945': 'Neutral',
 '1962': 'Neutral',
 '1h': 'Neutral',
 '2020': 'Neutral',
 '21h30': 'Neutral',
 '24': 'Neutral',
 '248': 'Neutral',
 '25k': 'Neutral',
 '28': 'Neutral',
 '2k4': 'Neutral',
 '2k5': 'Neutral',
 '30k': 'Neutral',
 '360': 'Neutral',
 '42': 'Neutral',
 '4k': 'Positive',
 '4tr': 'Neutral',
 '63': 'Positive',
 '8h30': 'Neutral',
 'acc': 'Neutral',
 'ace': 'Neutral',
 'anh_chị_em': 'Neutral',
 'bch': 'Neutral',
 'bcs': 'Negative',
 'bcvt': 'Positive',
 'biến': 'Neutral',
 'bài_bản': 'Positive',
 'bài_tập_lớn': 'Neutral',
 'bày_tỏ': 'Neutral',
 'bán_nước': 'Negative',
 'báo_cáo': 'Positive',
 'bảo_hiểm_y_tế': 'Neutral',
 'bẩn': 'Negative',
 'bận': 'Neutral',
 'bổ_ích': 'Positive',
 'bỡ_ngỡ': 'Neutral',
 'bức_xúc': 'Negative',
 'cao_cấp': 'Neutral',
 'cb': 'Neutral',
 'center': 'Neutral',
 'cetank': 'Neutral',
 'chat': 'Neutral',
 'chatgpt':

In [394]:
print(feature_vocabulary_trs.get_feature_vocabulary("Negative"))

['bcs', 'bán_nước', 'bẩn', 'bức_xúc', 'chiếm', 'cư_xử', 'cất', 'cắt', 'cốp', 'deadline', 'dong', 'dí', 'giang_hồ', 'giáo_vụ', 'grab', 'gà', 'gậy', 'hình_như', 'khâu', 'lão', 'lùa', 'lũ', 'may_ra', 'ngoại_hình', 'ngón', 'nhan_sắc', 'nón', 'nọ', 'phản_ánh', 'soát', 'sập', 'thiếu_thốn', 'trườg', 'tệ', 'tổn_thương', 'web', 'xước', 'áo_mưa', 'ó', 'đơn_phương', 'đực', 'ốm', '1k', 'dắt', 'chửi', 'bốt', 'chặn', 'im', 'kêu', 'tắc', 'đếm', '2k', 'già', 'gọn', 'kẹt', 'phi', 'phụ', 'đổ']


Sử dụng phương pháp thêm từ đặc trưng để thêm các trọng số Positive, Negative và Neutral vào mỗi câu:

In [395]:
def add_postfix(sentences : list[str]) -> list[str]:
    adding = []
    for key in feature_vocabulary:
        adding += [feature_vocabulary[key]] * sentences.count(key)
    return sentences + adding

In [396]:
i = 98
print(dsf["Remove_stopword"].iloc[i])
add_postfix(dsf["Remove_stopword"].iloc[i])

['nhớ', 'chép', 'đầy_đủ', 'thầy', 'nói', 'chép', 'hết', 'không', 'bỏ', 'chữ']


['nhớ', 'chép', 'đầy_đủ', 'thầy', 'nói', 'chép', 'hết', 'không', 'bỏ', 'chữ']

In [397]:
dsf["important_feature"] = dsf["Remove_stopword"].apply(add_postfix)

In [398]:
dsf["important_feature"].head(10)

0    [tìm, chủ_nhân, chai, nước, kẹt, làm, anh_em, ...
1    [mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc...
2    [tôi, thật_sự, bất_ngờ, hoành_tráng, trường_họ...
3                                      [k62, giãy_nảy]
4    [tiếng, anh, thi, đầu_vào, không, bắt_buộc, đâ...
5            [rường, tặng, quà, tết, năm_ngoái, không]
6    [lớp, mình, còn, không, gái, toàn, vịt, đực, N...
7    [nghỉ, học, đi, em, trường, nổi, 9tr, 1thang, ...
8                    [thích, nhá, chuẩn_bị, chụp, ảnh]
9    [nhân_viên, trông, giữ, xe, tỏ, thái_độ, thu, ...
Name: important_feature, dtype: object

DÙNG WORD2VEC ĐỂ NHÚNG TỪ

In [399]:
vocal_file = dsf["important_feature"]

In [400]:
model_vocal = gensim.models.Word2Vec(
    vector_size=100,
    window = 6,
    min_count = 1,
    workers=4,
    epochs=25
)

03:09:41 gensim.utils INFO: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-05-13T03:09:41.644321', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]


In [401]:
model_vocal.build_vocab(vocal_file, progress_per=2000)

03:09:42 gensim.models.word2vec INFO: collecting all words and their counts
		at Line 582 [scan_vocab() in word2vec.py, word2vec]
03:09:42 gensim.models.word2vec INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:09:42 gensim.models.word2vec INFO: PROGRESS: at sentence #2000, processed 34759 words, keeping 4750 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:09:42 gensim.models.word2vec INFO: collected 5345 word types from a corpus of 44307 raw words and 2497 sentences
		at Line 588 [scan_vocab() in word2vec.py, word2vec]
03:09:42 gensim.models.word2vec INFO: Creating a fresh vocabulary
		at Line 637 [prepare_vocab() in word2vec.py, word2vec]
03:09:42 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 5345 unique words (100.00% of original 5345, drops 0)', 'datetime': '2023-05-13T03:09:42.093529', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6

In [402]:
model_vocal.train(vocal_file, total_examples=model_vocal.corpus_count, epochs = model_vocal.epochs)

03:09:42 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'training model with 4 workers on 5345 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=6 shrink_windows=True', 'datetime': '2023-05-13T03:09:42.470865', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:09:42 gensim.models.word2vec INFO: EPOCH 0: training on 44307 raw words (36206 effective words) took 0.0s, 1432103 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:09:42 gensim.models.word2vec INFO: EPOCH 1: training on 44307 raw words (36263 effective words) took 0.0s, 1542299 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:09:42 gensim.models.word2vec INFO: EPOCH 2: training on 44307 raw words (36229 effective words) took 0.0s, 1422916 effective words/s
	

(906019, 1107675)

In [403]:
model_vocal.wv[0]

array([-0.01845082,  1.0943823 , -0.3708153 ,  1.4482787 , -1.3958234 ,
       -2.0160134 ,  1.0231982 ,  1.4373963 , -1.642977  ,  0.16836911,
        0.4363697 , -0.6769406 ,  0.20362289,  1.0967828 , -0.3639581 ,
       -0.32662004,  0.28923467, -1.0303075 , -1.1090121 , -2.2601292 ,
        1.2512093 ,  1.2095017 ,  0.96741   , -0.87085867, -0.5428892 ,
       -1.1975913 , -0.15745977, -0.52407545, -1.1204085 , -0.77988434,
        0.46191177,  0.1699076 ,  1.7369497 , -1.5113394 , -0.3669423 ,
        1.682892  ,  0.74955994, -0.9115758 , -1.4727331 , -1.753378  ,
       -0.6765126 , -1.1561162 , -0.841597  , -0.24417275,  1.7337482 ,
        0.0617253 , -1.820804  ,  0.1551999 ,  1.7149467 ,  0.9286291 ,
       -0.49942115,  0.39559618, -0.22998117, -0.41601226, -0.41254953,
       -0.06375884,  1.1191938 , -1.2473893 , -2.0079956 ,  0.2267471 ,
        0.9010799 ,  0.10464333, -0.24415326, -0.81008184, -0.729119  ,
        1.180213  , -0.7677813 ,  1.6775981 , -1.2665923 ,  0.49

In [404]:
model_vocal.wv.most_similar("cô")

[('giọng', 0.9701904654502869),
 ('hương', 0.9545503258705139),
 ('tỉ_mỉ', 0.9513570666313171),
 ('giảng', 0.9495955109596252),
 ('thầy', 0.9475370049476624),
 ('khôg', 0.9407190084457397),
 ('lắm', 0.935977578163147),
 ('hiền', 0.9357029795646667),
 ('khó_tính', 0.9345839619636536),
 ('phát_biểu', 0.9334244728088379)]

In [405]:
model_vocal.save('model_vocal.bin')

03:09:44 gensim.utils INFO: Word2Vec lifecycle event {'fname_or_handle': 'model_vocal.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-13T03:09:44.389714', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:09:44 gensim.utils INFO: not storing attribute cum_table
		at Line 710 [_save_specials() in utils.py, utils]
03:09:44 gensim.utils INFO: saved model_vocal.bin
		at Line 618 [_smart_save() in utils.py, utils]


In [406]:
#  Tạo DataFrame từ các vecto đầu ra của các câu
df_vocal = pd.DataFrame(columns=[str(i) for i in range(100)])
sentences = dsf["important_feature"]
for sentence in sentences:
    sentence_vector = []
    for word in sentence:
        try:
            word_vector = model_vocal.wv[word]
            sentence_vector.append(word_vector)
        except KeyError:
            pass
    if sentence_vector:
        sentence_vector = pd.DataFrame(sentence_vector)
        sentence_vector = sentence_vector.mean()
        df_vocal = df_vocal.append(pd.Series(sentence_vector.values, index = df_vocal.columns), ignore_index=True)

C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\2982471069.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_vocal = df_vocal.append(pd.Series(sentence_vector.values, index = df_vocal.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\2982471069.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_vocal = df_vocal.append(pd.Series(sentence_vector.values, index = df_vocal.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\2982471069.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_vocal = df_vocal.append(pd.Series(sentence_vector.values, index = df_vocal.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\2982471069.py:15: FutureWarning:

In [407]:
df_vocal['Labels'] = dsf['Labels']

In [408]:
df_vocal

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Labels
0,-0.133999,0.124521,0.018237,-0.359622,0.353545,-0.391519,-0.034172,0.630012,0.019281,-0.176345,...,0.383327,0.097711,0.025961,0.557088,0.368474,0.471114,-0.321603,0.108427,-0.118465,Neutral
1,-0.108249,0.356784,0.038340,-0.050055,0.157825,-0.470025,0.075137,0.646143,-0.252109,-0.057615,...,0.143116,-0.025944,0.088018,0.618642,0.378198,0.430349,-0.308679,-0.158195,0.014888,Neutral
2,-0.291026,0.403947,-0.209878,0.109910,0.353283,-0.474878,-0.037458,0.443315,-0.386169,-0.094083,...,0.362480,0.029618,0.050864,0.588182,0.242363,0.621940,-0.129310,-0.289850,-0.277085,Positive
3,-0.034320,0.125062,-0.083733,-0.145711,0.130983,-0.217969,0.052818,0.197816,-0.033735,-0.082355,...,0.151712,-0.028842,0.087964,0.217231,0.117141,0.194091,-0.135054,0.017734,-0.095053,Negative
4,-0.063896,0.252754,-0.344221,-0.126667,0.137826,-0.472645,-0.113708,0.176833,-0.046207,-0.013810,...,0.446927,-0.220192,0.143710,0.415934,0.147800,0.465009,0.056006,-0.122396,-0.354254,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,0.024714,0.418212,-0.345346,-0.299509,0.264791,-0.467731,0.372170,0.807207,-0.374932,0.286044,...,0.274725,0.027247,0.164123,1.008613,0.523378,0.084920,-0.617286,0.128605,-0.148878,Positive
2492,-0.063716,0.459389,-0.279347,0.264911,-0.076201,-0.653945,0.267222,0.344995,-0.480029,0.045995,...,0.332874,-0.157204,0.296142,0.738957,0.303925,0.278012,-0.279164,-0.118823,-0.134231,Neutral
2493,-0.051534,0.345844,-0.933171,-0.862183,0.634136,-0.380133,0.240208,0.250587,-0.050015,-0.041987,...,0.374148,0.016147,0.524998,0.336303,0.245443,0.590268,-0.291210,-0.029971,-0.312471,Neutral
2494,-0.016942,0.166110,-0.098173,-1.078645,0.593961,-0.551812,0.240078,0.649149,0.226403,-0.424686,...,0.435721,0.109633,0.321294,0.475418,0.331054,0.494325,-0.451015,0.224621,-0.124775,Positive


In [409]:
df_vocal.to_excel('vocal_result.xlsx', index=False)

## Dùng từ điển có từ dừng 

In [57]:
from libs.utils import *

In [58]:
print(dsf.iloc[0])
type(dsf["Repair_words"].iloc[0])

Opinion                   Tạch 5 lần GT1 có coi là kỉ lục không
Labels                                                  Neutral
Remove_endl               Tạch 5 lần GT1 có coi là kỉ lục không
Remove_punc               Tạch 5 lần GT1 có coi là kỉ lục không
Remove_emoji              Tạch 5 lần GT1 có coi là kỉ lục không
Remove_space              Tạch 5 lần GT1 có coi là kỉ lục không
Lower                     tạch 5 lần gt1 có coi là kỉ lục không
Join_word       [tạch, 5, lần, gt1, có, coi, là, kỉ_lục, không]
Repair_words    [tạch, 5, lần, gt1, có, coi, là, kỉ_lục, không]
Name: 0, dtype: object


list

In [59]:
dsf[{"Labels", "Repair_words"}]

C:\Users\fpt\AppData\Local\Temp\ipykernel_11332\1581093220.py:1: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  dsf[{"Labels", "Repair_words"}]


,Labels,Repair_words
0,Neutral,"[tạch, 5, lần, gt1, có, coi, là, kỉ_lục, không]"
1,Negative,"[ám_ảnh, đầu, năm, nhất, học, chính_trị, pháp_..."
2,Neutral,"[đk, ký_túc_xá, chưa, xin, được, giấy, khai_bá..."
3,Negative,"[thiếu, combo, mất, nón, bh, ở, cs1]"
4,Negative,"[chưa, cảm_thấy, thực_sự, hài_lòng, về, cách, ..."
...,...,...
2492,Neutral,"[chả, có, gì, mà, chanh, cái, cả, cả, 2, ông, ..."
2493,Positive,"[dễ_tính, nhé]"
2494,Neutral,"[nên, phân_biệt, rõ_ràng, giữa, giao_thông_vận..."
2495,Neutral,"[học, xưởng, tâm_trí, trên, mây]"


In [60]:
corpus: list[list[str]] = [doc for doc in dsf["Repair_words"]]
labels: list[str] = dsf["Labels"].to_list()

In [61]:
from libs.feature_vocabulary import feature_vocabulary_transfer
feature_vocabulary_trs = feature_vocabulary_transfer("gini")
feature_vocabulary = feature_vocabulary_trs.fit_transform(corpus, labels, 0.3, 3)

In [62]:
feature_vocabulary

{'00': 'Neutral',
 '12h': 'Neutral',
 '14': 'Neutral',
 '14h30': 'Neutral',
 '17h30': 'Neutral',
 '18h': 'Neutral',
 '18h30': 'Neutral',
 '1945': 'Neutral',
 '1962': 'Neutral',
 '1h': 'Neutral',
 '2020': 'Neutral',
 '21h30': 'Neutral',
 '24': 'Neutral',
 '248': 'Neutral',
 '25k': 'Neutral',
 '28': 'Neutral',
 '2k4': 'Neutral',
 '2k5': 'Neutral',
 '30k': 'Neutral',
 '360': 'Neutral',
 '42': 'Neutral',
 '4k': 'Positive',
 '4tr': 'Neutral',
 '63': 'Positive',
 '8h30': 'Neutral',
 'acc': 'Neutral',
 'ace': 'Neutral',
 'anh_chị_em': 'Neutral',
 'bch': 'Neutral',
 'bcs': 'Negative',
 'bcvt': 'Positive',
 'biến': 'Neutral',
 'bài_bản': 'Positive',
 'bài_tập_lớn': 'Neutral',
 'bày_tỏ': 'Neutral',
 'bán_nước': 'Negative',
 'báo_cáo': 'Positive',
 'bảo_hiểm_y_tế': 'Neutral',
 'bẩn': 'Negative',
 'bận': 'Neutral',
 'bổ_ích': 'Positive',
 'bỡ_ngỡ': 'Neutral',
 'bức_xúc': 'Negative',
 'cao_cấp': 'Neutral',
 'cb': 'Neutral',
 'center': 'Neutral',
 'cetank': 'Neutral',
 'chat': 'Neutral',
 'chatgpt':

In [63]:
save_model("feature_vocabulary.pkl", feature_vocabulary)

21:35:33 root INFO: Đang lưu lại mô hình...
		at Line 88 [save_model() in utils.py, utils]
21:35:34 root INFO: Lưu thành công!
		at Line 92 [save_model() in utils.py, utils]


In [416]:
print(feature_vocabulary_trs.get_feature_vocabulary("Negative"))

['bcs', 'bán_nước', 'bẩn', 'bức_xúc', 'chiếm', 'cmn', 'cư_xử', 'cất', 'cắt', 'cốp', 'deadline', 'dong', 'dí', 'giang_hồ', 'giáo_vụ', 'grab', 'gà', 'gậy', 'hình_như', 'khâu', 'lão', 'lùa', 'lũ', 'may_ra', 'ngoại_hình', 'ngón', 'nhan_sắc', 'nón', 'nọ', 'phản_ánh', 'soát', 'sập', 'thiếu_thốn', 'trườg', 'tệ', 'tổn_thương', 'web', 'xước', 'áo_mưa', 'ó', 'đơn_phương', 'đực', 'ốm', '1k', 'dắt', 'chửi', 'bốt', 'chặn', 'im', 'kêu', 'tắc', 'đếm', '2k', 'già', 'gọn', 'kẹt', 'phi', 'phụ', 'đổ']


In [417]:
def add_postfix(sentences : list[str]) -> list[str]:
    adding = []
    for key in feature_vocabulary:
        adding += [feature_vocabulary[key]] * sentences.count(key)
    return sentences + adding

In [418]:
i = 98
print(dsf["Repair_words"].iloc[i])
add_postfix(dsf["Repair_words"].iloc[i])

['nhớ', 'chép', 'đầy_đủ', 'nhé', 'thầy', 'nói', 'gì', 'chép', 'hết', 'không', 'được', 'bỏ', 'chữ', 'nào']


['nhớ',
 'chép',
 'đầy_đủ',
 'nhé',
 'thầy',
 'nói',
 'gì',
 'chép',
 'hết',
 'không',
 'được',
 'bỏ',
 'chữ',
 'nào']

In [419]:
dsf["important_feature_sw"] = dsf["Repair_words"].apply(add_postfix)

In [420]:
dsf["important_feature_sw"].head(10)

0    [đã, tìm, ra, những, chủ_nhân, của, chai, nước...
1    [mọi, người, nhớ, đeo, khẩu_trang, uống, thuốc...
2    [tôi, thật_sự, bất_ngờ, trước, sự, hoành_tráng...
3                                      [k62, giãy_nảy]
4    [tiếng, anh, thi, đầu_vào, là, không, bắt_buộc...
5    [rường, có_thể, tặng, quà, tết, như, năm_ngoái...
6    [lớp, mình, còn, không, có, gái, toàn, vịt, đự...
7    [nghỉ, học, đi, em, ra, trường, có, nổi, 9tr, ...
8              [thích, nhá, chuẩn_bị, được, chụp, ảnh]
9    [nhân_viên, trông, giữ, xe, tỏ, thái_độ, thu, ...
Name: important_feature_sw, dtype: object

Nhúng từ bằng Word2Vec

In [421]:
vocal_files = dsf["important_feature_sw"]

In [422]:
models_vocal = gensim.models.Word2Vec(
    vector_size=100,
    window = 6,
    min_count = 1,
    workers=4,
    epochs=20
)

03:10:05 gensim.utils INFO: Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=100, alpha=0.025>', 'datetime': '2023-05-13T03:10:05.671564', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]


In [423]:
models_vocal.build_vocab(vocal_files, progress_per=2000)

03:10:05 gensim.models.word2vec INFO: collecting all words and their counts
		at Line 582 [scan_vocab() in word2vec.py, word2vec]
03:10:05 gensim.models.word2vec INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:10:05 gensim.models.word2vec INFO: PROGRESS: at sentence #2000, processed 45916 words, keeping 4873 word types
		at Line 565 [_scan_vocab() in word2vec.py, word2vec]
03:10:05 gensim.models.word2vec INFO: collected 5471 word types from a corpus of 58735 raw words and 2497 sentences
		at Line 588 [scan_vocab() in word2vec.py, word2vec]
03:10:05 gensim.models.word2vec INFO: Creating a fresh vocabulary
		at Line 637 [prepare_vocab() in word2vec.py, word2vec]
03:10:05 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'effective_min_count=1 retains 5471 unique words (100.00% of original 5471, drops 0)', 'datetime': '2023-05-13T03:10:05.832352', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6

In [424]:
models_vocal.train(vocal_files, total_examples=models_vocal.corpus_count, epochs = models_vocal.epochs)

03:10:05 gensim.utils INFO: Word2Vec lifecycle event {'msg': 'training model with 4 workers on 5471 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=6 shrink_windows=True', 'datetime': '2023-05-13T03:10:05.980563', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'train'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:10:06 gensim.models.word2vec INFO: EPOCH 0: training on 58735 raw words (47527 effective words) took 0.1s, 579996 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:10:06 gensim.models.word2vec INFO: EPOCH 1: training on 58735 raw words (47522 effective words) took 0.1s, 321582 effective words/s
		at Line 1652 [_log_epoch_end() in word2vec.py, word2vec]
03:10:06 gensim.models.word2vec INFO: EPOCH 2: training on 58735 raw words (47498 effective words) took 0.1s, 864411 effective words/s
		at

(951301, 1174700)

In [425]:
models_vocal.wv[0]

array([ 0.11703061, -0.17694885, -0.17303684,  0.26029143, -0.80167544,
       -1.1972871 ,  1.5806664 ,  2.080378  , -0.4194201 , -1.1547773 ,
        0.5608865 , -0.96855944, -0.7765041 ,  0.1449222 ,  0.08509538,
       -1.4198945 , -0.9837195 , -2.032321  , -0.0927316 , -0.8585526 ,
        1.0090072 ,  1.171657  ,  0.38040325,  0.4973725 , -0.16583692,
       -0.22780663, -0.4004066 , -1.0583979 , -0.12573037, -0.9030092 ,
        0.4069924 ,  0.51979417,  1.207837  , -0.6379531 , -1.0445865 ,
        0.6605863 , -0.41231522, -0.33137658, -1.4917405 , -2.1553507 ,
       -0.8529172 ,  0.26864958, -0.08138349, -0.1770883 ,  1.1152232 ,
       -0.64902896, -1.2004983 ,  1.8392283 ,  0.87584305,  1.7337242 ,
       -0.29382655,  0.23429172,  0.16171607, -0.2306146 ,  0.07971669,
        0.1928572 ,  0.64782536, -1.1624817 , -0.3180468 , -0.21457148,
        1.1923057 ,  0.08447551,  0.71228373, -0.39037108, -1.404763  ,
        0.41320476, -0.5232079 ,  1.842921  , -1.1910573 ,  0.63

In [426]:
models_vocal.wv.most_similar("cô")

[('thầy', 0.972285807132721),
 ('dễ', 0.9506537318229675),
 ('giảng', 0.9375772476196289),
 ('tỉ_mỉ', 0.9307723641395569),
 ('nói_tục', 0.9248530268669128),
 ('hiền', 0.9200526475906372),
 ('giọng', 0.920035719871521),
 ('dạy', 0.9190476536750793),
 ('uầy', 0.9121481776237488),
 ('ngặt', 0.9029234647750854)]

In [427]:
models_vocal.wv.similarity(w1 = "trọ", w2 = "thầy")

0.36054075

In [428]:
models_vocal.wv.similarity(w1 = "trọ", w2 = "ký_túc_xá")

0.8601792

In [429]:
models_vocal.save('model_vocal_sw.bin')

03:10:07 gensim.utils INFO: Word2Vec lifecycle event {'fname_or_handle': 'model_vocal_sw.bin', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2023-05-13T03:10:07.894628', 'gensim': '4.3.1', 'python': '3.10.10 (tags/v3.10.10:aad5f6a, Feb  7 2023, 17:20:36) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'saving'}
		at Line 448 [add_lifecycle_event() in utils.py, utils]
03:10:07 gensim.utils INFO: not storing attribute cum_table
		at Line 710 [_save_specials() in utils.py, utils]
03:10:07 gensim.utils INFO: saved model_vocal_sw.bin
		at Line 618 [_smart_save() in utils.py, utils]


In [430]:
#  Tạo DataFrame từ các vecto đầu ra của các câu
df_vocal_sw = pd.DataFrame(columns=[str(i) for i in range(100)])
sentences = dsf["important_feature_sw"]
for sentence in sentences:
    sentence_vector = []
    for word in sentence:
        try:
            word_vector = models_vocal.wv[word]
            sentence_vector.append(word_vector)
        except KeyError:
            pass
    if sentence_vector:
        sentence_vector = pd.DataFrame(sentence_vector)
        sentence_vector = sentence_vector.mean()
        df_vocal_sw = df_vocal_sw.append(pd.Series(sentence_vector.values, index = df_vocal_sw.columns), ignore_index=True)

C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\4231180959.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_vocal_sw = df_vocal_sw.append(pd.Series(sentence_vector.values, index = df_vocal_sw.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\4231180959.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_vocal_sw = df_vocal_sw.append(pd.Series(sentence_vector.values, index = df_vocal_sw.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\4231180959.py:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_vocal_sw = df_vocal_sw.append(pd.Series(sentence_vector.values, index = df_vocal_sw.columns), ignore_index=True)
C:\Users\fpt\AppData\Local\Temp\ipykernel_7196\42311

In [431]:
df_vocal_sw['Labels'] = dsf['Labels']

In [432]:
df_vocal_sw

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,Labels
0,-0.017220,0.277157,-0.111707,-0.349179,0.366307,-0.455927,0.011527,0.619228,-0.265354,-0.077656,...,0.182200,0.126116,-0.088191,0.393318,0.261711,0.226152,-0.011909,0.140643,-0.530037,Neutral
1,-0.153917,0.259767,-0.002951,-0.173699,0.197399,-0.578599,0.060220,0.747914,-0.291067,-0.132252,...,0.162456,-0.117789,-0.047863,0.481066,0.384776,0.286126,-0.102757,0.058884,-0.341421,Neutral
2,-0.296557,0.141795,-0.202536,-0.095218,0.490493,-0.644022,0.221975,0.718072,-0.482908,-0.216394,...,0.224070,-0.097153,-0.175697,0.442631,0.738762,0.704702,-0.324776,0.141180,-0.605276,Positive
3,-0.075159,0.112620,0.017908,-0.083063,-0.007767,-0.203668,0.025163,0.282430,-0.138269,-0.017422,...,0.052961,0.046924,0.015253,0.209211,0.115370,0.088917,-0.017441,0.078322,-0.102974,Negative
4,-0.210828,0.035464,-0.059586,-0.224863,0.141136,-0.418833,0.220955,0.379102,-0.379969,-0.152038,...,0.145195,0.131024,0.091785,0.417351,0.459434,0.395014,-0.134563,0.304424,-0.504236,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,-0.057981,0.167438,-0.284758,-0.541417,0.342441,-0.464307,0.198545,0.762128,-0.090990,-0.147798,...,0.239817,-0.135956,0.140075,0.501787,0.392524,0.384588,0.252567,0.400677,-0.632058,Neutral
2493,-0.187317,0.177853,-0.223015,-0.248474,0.014659,-0.534343,0.116825,0.758176,-0.335731,-0.181081,...,0.300250,-0.090224,0.111604,0.557962,0.455144,0.367224,-0.059533,0.325908,-0.521351,Neutral
2494,-0.283796,0.421815,-0.284590,-0.823916,0.251688,-0.549902,-0.263467,0.654195,-0.230282,0.109601,...,0.386258,-0.253285,0.008639,0.417933,0.156734,0.656569,0.412077,0.261403,-0.626197,Positive
2495,-0.235899,0.450338,0.058970,-0.562637,0.025124,-0.668665,-0.311251,0.488503,-0.238801,0.154123,...,0.304791,0.022149,0.015535,0.445687,-0.179286,0.078765,0.264759,0.112201,-0.181629,Negative


In [433]:
df_vocal_sw.to_excel('vocal_sw_result.xlsx', index=False)